In [1]:
import numpy as np 
import numpy.ma as ma

import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib

import datetime
from tqdm import tqdm

from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.table import Table

from ResidualUtils import getwavs, getspecpath, getapstarpath, create_mask, get_selected_stars, get_residual, get_selected_stars

# Overview
To study the spectral features of the ISM in observed spectra, we need a reliable characterization of the spectral features of those background objects. We've produced a catalog of the median stellar residuals between APOGEE spectra and their corresponding ASPCAP fits, which can serve as a model of their typical deviations.

To produce our catalog, we grouped `ASPCAPSTAR` spectra by `TEFF`, `LOGG`, and `M_H` using the binning scheme described in [Ashok et al. 2021](https://doi.org/10.3847/1538-3881/abd7f1). After applying `APSTAR/ASSTAR` bitmasks and our own supplementary masks to remove pixels with high uncertainties, we calculate the median residual 
$$\langle R\rangle = \text{median}(F_\lambda / F_{\lambda}')$$
for $F_\lambda$ the combined stellar spectrum and $F_{\lambda}'$ the corresponding ASPCAP model.



In [2]:
tabfile = '/uufs/chpc.utah.edu/common/home/sdss/dr17/apogee/spectro/aspcap/dr17/synspec_rev1/allStarLite-dr17-synspec_rev1.fits'
stars = Table.read(tabfile, hdu = 1)

CA_fields = ['CA_161-06_btx', 'CA_161-06_btx', 'CA_162-11_btx', 'CA_163-05_btx', 'CA_163-08_btx',
             'CA_165-07_btx', 'CA_165-10_btx']
CA_inds = np.unique(np.concatenate([np.where(stars['FIELD'] == field)[0] for field in CA_fields]))
CA_stars = stars[CA_inds]
CA_clean = CA_stars[np.invert(np.logical_or.reduce([np.isnan(CA_stars['TEFF']), np.isnan(CA_stars['LOGG']), 
                                           np.isnan(CA_stars['M_H'])]))]

select_bitmasked = np.where((np.bitwise_and(stars['ASPCAPFLAG'], 23) == 0) | 
                            (np.bitwise_and(stars['ASPCAPFLAG'], 19) == 0))[0] # perhaps I should also bitmask CA_clean

stars = stars[select_bitmasked]
stars = stars[np.where((stars['TEFF_ERR'] < 600) & (stars['LOGG_ERR'] < 0.5) & (stars['M_H_ERR'] < 0.3))[0]]
stars = stars[np.where(np.abs(stars['GLAT']) > 10)[0]]

/uufs/chpc.utah.edu/sys/installdir/python/3.6.3/lib/python3.6/site-packages/astropy/table/column.py:1020: RuntimeWarning: invalid value encountered in less
  result = getattr(super(), op)(other)
